In [6]:
# Dependencies
import pandas as pd
import requests as req
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver

In [7]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [8]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# Step 1 - Scraping

# NASA Mars News

In [9]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
# Assign the text to variables that you can reference later.
# Visit url for NASA Mars News -- Latest News
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, "html.parser")

# Extract article title and paragraph text
article = soup.find("div", class_='list_text')
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_ ="article_teaser_body").text
print(news_title)
print(news_p)

The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. 


# JPL Mars Space Images - Featured Image

In [23]:
# Visit url for JPL Featured Space Image
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

# Go to 'FULL IMAGE'
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

# Go to 'more info'
browser.click_link_by_partial_text('more info')

# Parse HTML with Beautiful Soup
html = browser.html
image_soup = bs(html, 'html.parser')

# Scrape the URL
feat_img_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://www.jpl.nasa.gov{feat_img_url}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17845_hires.jpg


# Mars Weather

In [24]:
# Visit Twitter url for latest Mars Weather
tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(tweet_url)
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Extract latest tweet
tweet_container = soup.find_all('div', class_="js-tweet-text-container")

# Loop through latest tweets and find the tweet that has weather information
for tweet in tweet_container: 
    mars_weather = tweet.find('p').text
    if 'sol' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

# Mars Facts

In [25]:
# Visit Mars Facts webpage for interesting facts about Mars
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
html = browser.html

In [31]:
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

# Use Pandas to "read_html" to parse the URL
tables = pd.read_html(facts_url)

#Find Mars Facts DataFrame in the lists of DataFrames
df = tables[0]

#Assign the columns
df.columns = ['Description', 'Values']
html_table = df.to_html(table_id="html_tbl_css",justify='left',index=False)
data = df.to_dict(orient='records')
df

,Description,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [32]:
# Use Pandas to convert the data to a HTML table string
mars_facts.to_html('table.html')

# Mars Hemispheres

In [33]:
# Visit USGS webpage for Mars hemispehere images
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, "html.parser")

# Create dictionary to store titles & links to images
hemisphere_image_urls = []

# Retrieve all elements that contain image information
results = soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

# Iterate through each image
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})

# Print image title and url
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
